# Tutorial: Build a regression model with Open Datasets

In this tutorial, you leverage the convenience of Azure Open Datasets to create a regression model to predict NYC taxi fare prices. Easily download publicly available taxi, holiday and weather data to create a dataset that can train a regression model using sklearn.

In [3]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-telemetry 1.39.0 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-telemetry~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataset-runtime 1.39.0 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-dataset-runtime~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0.post1 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_

## Download Data
Begin by downloading the NYC Taxi dataset from Azure Open Datasets. In non-Spark environments, Open Datasets only allows one month of data at a time with certain classes to avoid MemoryError with large datasets. To download 1 year of taxi data, we will fetch 2000 random samples from each month.

Note: Open Datasets has mirroring classes for working in Spark where data size and memory are not a concern.

In [4]:
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

green_taxi_df = pd.concat([NycTlcGreen(start + relativedelta(months=x), end + relativedelta(months=x)) \
        .to_pandas_dataframe().sample(2000) for x in range(12)])
green_taxi_df

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpcb14gde_\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=1\part-00119-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2689-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpczbth1la\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=2\part-00060-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2630-2.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpktzngm24\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=3\part-00196-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2766-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp_x_kkhyx\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=4\part-00121-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2691-1.c000.snappy.parquet
[Info] read from C:\

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
1290595,2,2016-01-02 22:49:49,2016-01-03 22:35:49,1,4.66,None,None,-73.954079,40.742111,-73.964226,...,1,17.5,0.5,0.5,0.3,3.76,0.00,NaN,22.56,1.0
449515,2,2016-01-08 11:46:07,2016-01-08 12:08:57,1,8.43,None,None,-73.982323,40.686008,-73.953018,...,1,26.0,0.0,0.5,0.3,5.36,0.00,NaN,32.16,1.0
1126491,1,2016-01-07 17:30:46,2016-01-07 17:34:53,1,0.80,None,None,-73.976128,40.684750,-73.972816,...,1,5.0,1.0,0.5,0.3,0.00,0.00,NaN,6.80,1.0
1396384,1,2016-01-15 16:28:41,2016-01-15 16:33:14,1,1.10,None,None,-73.963852,40.710499,-73.955399,...,1,5.5,1.0,0.5,0.3,1.08,0.00,NaN,8.38,1.0
1314979,2,2016-01-03 12:39:13,2016-01-03 13:08:30,3,5.03,None,None,-73.958130,40.648117,-73.882805,...,1,22.5,0.0,0.5,0.3,0.00,0.00,NaN,23.30,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72780,2,2016-12-06 02:11:34,2016-12-06 02:16:27,1,1.09,37,36,NaN,NaN,NaN,...,1,6.0,0.5,0.5,0.3,0.02,0.00,NaN,7.32,1.0
1003558,1,2016-12-25 01:28:24,2016-12-25 01:35:45,1,1.70,7,129,NaN,NaN,NaN,...,2,8.0,0.5,0.5,0.3,0.00,0.00,NaN,9.30,1.0
75863,2,2016-12-06 08:32:41,2016-12-06 09:18:17,1,8.66,17,233,NaN,NaN,NaN,...,2,35.0,0.0,0.0,0,0.00,0.00,NaN,35.00,2.0
1071556,2,2016-12-01 15:58:35,2016-12-01 17:06:07,5,19.69,116,132,NaN,NaN,NaN,...,1,52.0,0.0,0.5,0.3,11.67,5.54,NaN,70.01,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day. From those, we calculate the sin and cosine transformations to capture the cyclical nature of the variable which will allow the model to factor in time-based seasonality. This function also adds a static feature for the country code to join the holiday data. Use the apply() function on the dataframe to interatively apply this function to each row in the dataframe.

In [5]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    country_code = "US"
    hr_sin = np.sin(hour_of_day*(2.*np.pi/24))
    hr_cos = np.cos(hour_of_day*(2.*np.pi/24))
    dy_sin = np.sin(day_of_week*(2.*np.pi/7))
    dy_cos = np.cos(day_of_week*(2.*np.pi/7))
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day, country_code, hr_sin, hr_cos, dy_sin, dy_cos))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day", "country_code", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,tripType,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos
1290595,2,2016-01-02 22:49:49,2016-01-03 22:35:49,1,4.66,None,None,-73.954079,40.742111,-73.964226,...,1.0,1,2,5,22,US,-5.000000e-01,8.660254e-01,-0.974928,-0.222521
449515,2,2016-01-08 11:46:07,2016-01-08 12:08:57,1,8.43,None,None,-73.982323,40.686008,-73.953018,...,1.0,1,8,4,11,US,2.588190e-01,-9.659258e-01,-0.433884,-0.900969
1126491,1,2016-01-07 17:30:46,2016-01-07 17:34:53,1,0.80,None,None,-73.976128,40.684750,-73.972816,...,1.0,1,7,3,17,US,-9.659258e-01,-2.588190e-01,0.433884,-0.900969
1396384,1,2016-01-15 16:28:41,2016-01-15 16:33:14,1,1.10,None,None,-73.963852,40.710499,-73.955399,...,1.0,1,15,4,16,US,-8.660254e-01,-5.000000e-01,-0.433884,-0.900969
1314979,2,2016-01-03 12:39:13,2016-01-03 13:08:30,3,5.03,None,None,-73.958130,40.648117,-73.882805,...,1.0,1,3,6,12,US,1.224647e-16,-1.000000e+00,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72780,2,2016-12-06 02:11:34,2016-12-06 02:16:27,1,1.09,37,36,NaN,NaN,NaN,...,1.0,12,6,1,2,US,5.000000e-01,8.660254e-01,0.781831,0.623490
1003558,1,2016-12-25 01:28:24,2016-12-25 01:35:45,1,1.70,7,129,NaN,NaN,NaN,...,1.0,12,25,6,1,US,2.588190e-01,9.659258e-01,-0.781831,0.623490
75863,2,2016-12-06 08:32:41,2016-12-06 09:18:17,1,8.66,17,233,NaN,NaN,NaN,...,2.0,12,6,1,8,US,8.660254e-01,-5.000000e-01,0.781831,0.623490
1071556,2,2016-12-01 15:58:35,2016-12-01 17:06:07,5,19.69,116,132,NaN,NaN,NaN,...,1.0,12,1,3,15,US,-7.071068e-01,-7.071068e-01,0.433884,-0.900969


Remove some of the columns that you won't need for modeling or additional feature building. Rename the time field for pickup time, and additionally convert the time to midnight using `pandas.Series.dt.normalize`. This is done to all time features so that the datetime column can be later used as a key when joining datasets together at a daily level of granularity.

In [6]:
columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"]

green_taxi_df.drop(columns_to_remove, axis=1, inplace=True)

green_taxi_df["datetime"] = green_taxi_df["lpepPickupDatetime"].dt.normalize()
green_taxi_df.head(5)

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime
1290595,2,2016-01-02 22:49:49,1,4.66,-73.954079,40.742111,-73.964226,40.685703,22.56,1,2,5,22,US,-5.000000e-01,0.866025,-0.974928,-0.222521,2016-01-02
449515,2,2016-01-08 11:46:07,1,8.43,-73.982323,40.686008,-73.953018,40.771954,32.16,1,8,4,11,US,2.588190e-01,-0.965926,-0.433884,-0.900969,2016-01-08
1126491,1,2016-01-07 17:30:46,1,0.80,-73.976128,40.684750,-73.972816,40.692970,6.80,1,7,3,17,US,-9.659258e-01,-0.258819,0.433884,-0.900969,2016-01-07
1396384,1,2016-01-15 16:28:41,1,1.10,-73.963852,40.710499,-73.955399,40.722878,8.38,1,15,4,16,US,-8.660254e-01,-0.500000,-0.433884,-0.900969,2016-01-15
1314979,2,2016-01-03 12:39:13,3,5.03,-73.958130,40.648117,-73.882805,40.648281,23.30,1,3,6,12,US,1.224647e-16,-1.000000,-0.781831,0.623490,2016-01-03


### Enrich with Holiday Data

Now that the taxi data is downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. The holiday dataset is relatively small, so fetch the full set by using the `PublicHolidays` class constructor with no parameters for filtering. Preview the data to check the format.

In [7]:
from azureml.opendatasets import PublicHolidays

# call default constructor to download full dataset
holidays_df = PublicHolidays().to_pandas_dataframe()
holidays_df.head(5)

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpwks3ef15\https%3A\%2Fazureopendatastorage.azurefd.net\holidaydatacontainer\Processed\part-00000-tid-8468414522853579044-35925ba8-a227-4b80-9c89-17065e7bf1db-649-c000.snappy.parquet


,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date
19375,Argentina,Año Nuevo [New Year's Day],Año Nuevo [New Year's Day],None,AR,2008-01-01
19376,Australia,New Year's Day,New Year's Day,None,AU,2008-01-01
19377,Austria,Neujahr,Neujahr,None,AT,2008-01-01
19378,Belarus,Новый год,Новый год,None,BY,2008-01-01
19379,Belgium,Nieuwjaarsdag,Nieuwjaarsdag,None,BE,2008-01-01


Rename the `countryRegionCode` and `date` columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. Next, join the holiday data with the taxi data by performing a left-join using the Pandas `merge()` function. This will preserve all records from `green_taxi_df`, but add in holiday data where it exists for the corresponding `datetime` and `country_code`, which in this case is always `\"US\"`. Preview the data to verify that they were merged correctly.

In [8]:
holidays_df = holidays_df.rename(columns={"countryRegionCode": "country_code"})
holidays_df["datetime"] = holidays_df["date"].dt.normalize()

holidays_df.drop(["countryOrRegion", "holidayName", "date"], axis=1, inplace=True)

taxi_holidays_df = pd.merge(green_taxi_df, holidays_df, how="left", on=["datetime", "country_code"])
taxi_holidays_df[taxi_holidays_df["normalizeHolidayName"].notnull()]

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,...,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime,normalizeHolidayName,isPaidTimeOff
13,2,2016-01-01 02:07:43,1,0.98,-73.937721,40.848175,-73.923752,40.843361,6.80,1,...,4,2,US,0.500000,8.660254e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
20,1,2016-01-01 04:51:53,2,1.30,-73.893501,40.859203,-73.902733,40.842697,8.00,1,...,4,4,US,0.866025,5.000000e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
34,2,2016-01-01 03:02:24,1,3.94,-73.929543,40.849983,-73.866318,40.837284,14.30,1,...,4,3,US,0.707107,7.071068e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
42,2,2016-01-01 19:14:17,1,1.83,-73.979362,40.642715,-73.962181,40.660637,7.80,1,...,4,19,US,-0.965926,2.588190e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
48,2,2016-01-18 02:53:37,1,3.88,-73.917870,40.771244,-73.951935,40.731834,18.96,1,...,0,2,US,0.500000,8.660254e-01,0.000000,1.000000,2016-01-18,Martin Luther King Jr. Day,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23862,2,2016-12-26 21:54:41,1,2.27,NaN,NaN,NaN,NaN,10.80,12,...,0,21,US,-0.707107,7.071068e-01,0.000000,1.000000,2016-12-26,Christmas Day,True
23926,1,2016-12-25 23:17:07,1,0.50,NaN,NaN,NaN,NaN,6.35,12,...,6,23,US,-0.258819,9.659258e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True
23940,2,2016-12-26 11:10:39,1,6.37,NaN,NaN,NaN,NaN,30.41,12,...,0,11,US,0.258819,-9.659258e-01,0.000000,1.000000,2016-12-26,Christmas Day,True
23947,2,2016-12-26 18:50:42,1,1.84,NaN,NaN,NaN,NaN,13.71,12,...,0,18,US,-1.000000,-1.836970e-16,0.000000,1.000000,2016-12-26,Christmas Day,True


### Enrich with weather data

Now NOAA surface weather data can be appended to the taxi and holiday data. Use a similar approach to fetch the weather data by downloading one month at a time iteratively. Additionally, specify the `cols` parameter with an array of strings to filter the columns to download. This is a very large dataset containing weather surface data from all over the world, so before appending each month, filter the lat/long fields to near NYC using the `query()` function on the dataframe. This will ensure the `weather_df` doesn't get too large.

In [9]:
from azureml.opendatasets import NoaaIsdWeather
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

weather_df = pd.concat([NoaaIsdWeather(cols=["temperature", "precipTime", "precipDepth"], start_date=start + relativedelta(months=x), end_date=end + relativedelta(months=x))\
        .to_pandas_dataframe().query("latitude>=40.53 and latitude<=40.88 and longitude>=-74.09 and longitude<=-73.72 and temperature==temperature") for x in range(12)])

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpbfh17edt\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpbfh17edt\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpbfh17edt\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpbfh17edt\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00003-tid-738723883827836859-85e0759b-51fa-44

In [11]:
weather_df

,precipTime,latitude,precipDepth,datetime,usaf,temperature,longitude,wban
204647,NaN,40.783,NaN,2016-01-02 03:00:00,725030,2.8,-73.867,14732
204670,1.0,40.779,0.0,2016-01-22 13:51:00,725030,-4.4,-73.880,14732
204694,1.0,40.779,0.0,2016-01-08 02:51:00,725030,5.0,-73.880,14732
204701,1.0,40.779,0.0,2016-01-04 15:51:00,725030,-1.1,-73.880,14732
204715,1.0,40.779,0.0,2016-01-01 21:51:00,725030,4.4,-73.880,14732
...,...,...,...,...,...,...,...,...
1248471,1.0,40.789,0.0,2016-12-23 13:51:00,725053,4.4,-73.967,94728
1248555,1.0,40.789,0.0,2016-12-12 13:51:00,725053,5.0,-73.967,94728
1248580,NaN,40.789,NaN,2016-12-18 07:01:00,725053,3.9,-73.967,94728
1248597,1.0,40.789,0.0,2016-12-25 00:51:00,725053,7.8,-73.967,94728


Again call `pandas.Series.dt.normalize` on the `datetime` field in the weather data so it matches the time key in `taxi_holidays_df`.


Next group the weather data to have daily aggregated weather values. Define a dict `aggregations` to define how to aggregate each field at a daily level. For`temperature` take the mean and for `precipTime` and `precipDepth` take the daily maximum. Use the `groupby()` function along with the aggregations to group the data. Preview the data to ensure there is one record per day.

In [10]:
weather_df["datetime"] = weather_df["datetime"].dt.normalize()

# group by datetime
aggregations = {"precipTime": "max", "temperature": "mean", "precipDepth": "max"}
weather_df_grouped = weather_df.groupby("datetime").agg(aggregations)
weather_df_grouped.head(10)

,precipTime,temperature,precipDepth
datetime,,,
2016-01-01,1.0,5.197345,0.0
2016-01-02,1.0,2.567857,0.0
2016-01-03,1.0,3.846429,0.0
2016-01-04,1.0,0.123894,0.0
2016-01-05,6.0,-7.206250,0.0
2016-01-06,6.0,-0.896396,0.0
2016-01-07,6.0,3.180645,0.0
2016-01-08,1.0,4.384091,0.0
2016-01-09,6.0,6.710274,3.0


Note: The examples in this tutorial merge data using Pandas functions and custom aggregations, but the Open Datasets SDK has classes designed to easily merge and enrich data sets. See the [notebook](https://github.com/Azure/OpenDatasetsNotebooks/blob/master/tutorials/data-join/04-nyc-taxi-join-weather-in-pandas.ipynb) for code examples of these design patterns.

### Cleanse data

Merge the existing taxi and holiday data with the new weather data. This time `datetime` is the only key, and again perform a left-join of the data. Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [11]:
taxi_holidays_weather_df = pd.merge(taxi_holidays_df, weather_df_grouped, how="left", on=["datetime"])
taxi_holidays_weather_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,24000.000000,24000.000000,24000.000000,12000.000000,12000.000000,12000.000000,12000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.791125,1.346458,2.796666,-73.837558,40.693471,-73.872887,40.711812,14.635204,6.500000,15.092667,3.243417,13.680833,-0.254485,-1.656533e-02,-0.081774,-0.057280,13.181292,13.882253,1056.732458
std,0.406513,1.000358,2.926397,2.698413,1.488080,2.134112,1.177242,11.828928,3.452124,8.519582,1.957313,6.678472,0.662720,7.041336e-01,0.712081,0.694992,10.355086,9.490372,2823.301174
min,1.000000,0.000000,0.000000,-74.305374,0.000000,-74.305374,0.000000,-200.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.020000,-73.959984,40.694931,-73.966969,40.695563,7.800000,3.750000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,1.000000,6.591071,0.000000
50%,2.000000,1.000000,1.810000,-73.945587,40.746548,-73.944729,40.746832,11.300000,6.500000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,13.125893,10.000000
75%,2.000000,1.000000,3.490000,-73.918350,40.802372,-73.911234,40.791154,17.750000,9.250000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.433884,0.623490,24.000000,22.944737,127.000000
max,2.000000,6.000000,45.920000,0.000000,40.911461,0.000000,41.033749,493.720000,12.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,31.303665,9999.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the same bounds you used for filtering weather data. The `tripDistance` field has some bad data, because the minimum value is negative. The `passengerCount` field has bad data as well, with the max value being 210 passengers. Lastly, the `totalAmount` field has negative values, which don't make sense in the context of our model.

Filter out these anomolies using query functions, and then remove the last few columns unnecesary for training.

Note: since a random sample of 2000 was taken for each month of the taxi data, the statistics may vary each time this is ran.

In [12]:
final_df = taxi_holidays_weather_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88 and \
                                           pickupLongitude>=-74.09 and pickupLongitude<=-73.72 and \
                                           tripDistance>0 and tripDistance<75 and \
                                           passengerCount>0 and passengerCount<100 and \
                                           totalAmount>0")

Call `describe()` again on the data to ensure cleansing worked as expected. The final data is prepared and cleansed, consisting of taxi, holiday, and weather data, and is ready to use for machine learning model training.

In [13]:
final_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000,1.176000e+04,11760.000000,11760.000000,11760.000000,11760.000000,11760.000000
mean,1.790221,1.358248,2.831497,-73.936552,40.747342,-73.916044,40.734936,14.553398,3.504592,14.979167,3.243027,13.748214,-0.253053,-1.846857e-02,-0.079413,-0.052024,11.922109,10.269461,206.214371
std,0.407168,1.015054,2.904904,0.041569,0.056758,1.181864,0.653288,10.496079,1.709136,8.501361,1.963599,6.648319,0.664406,7.030417e-01,0.713567,0.694185,10.175067,8.541049,1278.601577
min,1.000000,1.000000,0.010000,-74.083717,40.572948,-74.182915,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.050000,-73.960323,40.695019,-73.967501,40.695725,8.000000,2.000000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,1.000000,3.371429,0.000000
50%,2.000000,1.000000,1.890000,-73.946125,40.746468,-73.945164,40.746540,11.300000,4.000000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,10.168182,3.000000
75%,2.000000,1.000000,3.530000,-73.918928,40.801964,-73.911892,40.789967,17.300000,5.000000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,17.239744,41.000000
max,2.000000,6.000000,45.920000,-73.743454,40.879841,0.000000,41.033749,202.800000,6.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,26.524107,9999.000000


## Train a model

The data is ready to train a machine learning model.

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

### Training Function

Define a function that can be used to create a model pipeline that can be trained and then used for scoring. This pipeline has 2 steps: preprocessing and model training.

<b>Preprocessing Stages:</b>
The preprocessing step of the pipeline also has 2 stages, one for numerical features and one for categorical features.
For the numerical features, let's fill in any blanks with 0's. While the training data may not have any nulls in the these fields, future data that is scored may and this step will take care of those for us. Optionally, a scaler transformation could be added in this step as well. Similarly for the categorical variables, let's have the null values filled with "MISSING". Additionally to the categorical variables, these will need to be one hot encoded, so we will include that step in our pipeline.

<b>Model Training Stage:</b>
An input parameter will determine which type of model of train. Let's test out a linear regression and random forest model to start. 

The two steps are put together into the pipeline which is what the function is returning.

In [42]:
def createClassModel(algo_name, catg, nums):
  numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])
  
  categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="MISSING")), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
  
  preprocesser = ColumnTransformer(transformers=[('num', numeric_transformer, nums), ('cat', categorical_transformer, catg)])
  
  if algo_name == 'linear_regression':
    model = LinearRegression()
    #model = RidgeCV(alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000])
  elif algo_name == 'random_forest':
    model = RandomForestRegressor(n_estimators=200)
  else:
    pass
  ModelPipeline = Pipeline(steps=[('preprocessor', preprocesser), ("model", model)])
  return ModelPipeline

Let's define the arguments that will be passed to the function. `catg_cols` is a list of the categorical variables that will be transformed in our processing step. `num_cols` is a list of the numerical variables that will be transformed in our processing step. Let's define the target column as `label` so it can be used in future steps as well.

In [37]:
catg_cols = ["vendorID", "month_num", "day_of_month", "normalizeHolidayName", "isPaidTimeOff"]
num_cols = ["passengerCount", "tripDistance", "precipTime", "temperature", "precipDepth", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]
label = ["totalAmount"]

The training is ready to begin, but first, let's make sure that the categorical variables are strings in our dataframe to ensure no errors in our pipeline. 

Next, the data is split into training and test sets by using the `train_test_split()` function in the `scikit-learn` library. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random number generator, so that your train-test splits are deterministic.

The training will happen in the for loop so that both algorithms can be tested. The createClassModel funtion is called to retreive the pipeline that can then be trained using the training dataset. 

Once trained, the test dataset is then ran through the model to test the model's performance. Using various functions from sklearn.metrics, the R2 score, MAPE, and RMSE can be used to measure model performance.

In [43]:
# make sure categorical columns are strings
final_df[catg_cols] = final_df[catg_cols].astype("str")

# split data
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(label, axis=1), final_df[label], test_size=0.2, random_state=222)

# test 2 algorithms
for algorithmname in ["linear_regression", "random_forest"]:
    fitPipeline = createClassModel(algorithmname, catg_cols, num_cols) # get pipeline
    fitPipeline.fit(X_train, y_train.values.ravel())                   # fit pipeine

    y_pred = fitPipeline.predict(X_test)                               # score with fitted pipeline

    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(algorithmname)
    print("R2:", r2)
    print("MAPE:", mape)
    print("RMSE:", rmse)
    print()

linear_regression
R2: 0.8643864872866158
MAPE: 0.15191865516530284
RMSE: 4.39273478136152



In [33]:
from sklearn.model_selection import train_test_split, GridSearchCV
param_grid = {
    
    "model__alpha": [100,1000,10000,100000],
}

grid_search = GridSearchCV(fitPipeline, param_grid, cv=5)
grid_search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value=0,
                                                                                                        strategy='constant'))]),
                                                                         ['passengerCount',
                                                                          'tripDistance',
                                                                          'precipTime',
                                                                          'temperature',
                                                                          'precipDepth',
                                                 

In [34]:
grid_search.fit(X_train, y_train)

print("Best params:")
print(grid_search.best_params_)

Best params:
{'model__alpha': 100}


In [35]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.863
